In [1]:
import pandas as pd
import torch
import ast
import numpy as np
import math

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


## Read in Results from Each Epoch

In [2]:
epoch1 = pd.read_json('Epoch1Eval/eval_predictions.jsonl', lines = True)
epoch2 = pd.read_json('Epoch2Eval/eval_predictions.jsonl', lines = True)
epoch3 = pd.read_json('Epoch3Eval/eval_predictions.jsonl', lines = True)

In [3]:
epoch1

,premise,hypothesis,label,predicted_scores,predicted_label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1,"[-1.654245257377624, 3.060402870178222, -1.909...",1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2,"[-3.788978338241577, -1.726562023162841, 5.308...",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0,"[3.499536752700805, -0.48690515756607006, -3.4...",0
3,Children smiling and waving at camera,They are smiling at their parents,1,"[-1.132757067680358, 2.950945138931274, -2.234...",1
4,Children smiling and waving at camera,There are children present,0,"[2.903966903686523, -0.252786874771118, -3.141...",0
...,...,...,...,...,...
549362,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',2,"[-1.904241561889648, 2.173691034317016, -0.770...",1
549363,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",1,"[-1.067546486854553, 2.117156028747558, -1.398...",1
549364,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,1,"[-1.448815464973449, 3.084083080291748, -2.108...",1
549365,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,2,"[-4.197515964508057, -0.9621648788452141, 4.58...",2


In [4]:
epoch2

,premise,hypothesis,label,predicted_scores,predicted_label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1,"[-2.395722389221191, 3.979343891143799, -2.407...",1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2,"[-4.647353649139404, -1.3414146900177, 5.42427...",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0,"[3.075013160705566, -0.061564125120639, -3.537...",0
3,Children smiling and waving at camera,They are smiling at their parents,1,"[-1.761919975280761, 3.331791877746582, -2.118...",1
4,Children smiling and waving at camera,There are children present,0,"[3.656948566436767, -0.518254280090332, -3.563...",0
...,...,...,...,...,...
549362,Four dirty and barefooted children.,four kids won awards for 'cleanest feet',2,"[-3.755248785018921, -0.224017441272735, 3.133...",2
549363,Four dirty and barefooted children.,"four homeless children had their shoes stolen,...",1,"[-1.609236121177673, 2.903273344039917, -1.745...",1
549364,A man is surfing in a bodysuit in beautiful bl...,A man in a bodysuit is competing in a surfing ...,1,"[-2.238006114959717, 3.957268714904785, -2.503...",1
549365,A man is surfing in a bodysuit in beautiful bl...,A man in a business suit is heading to a board...,2,"[-4.258902072906494, -0.61960083246231, 4.0823...",2


In [5]:
#created combined dataset
combined_set = epoch1
combined_set = combined_set.rename(columns = {'predicted_scores': 'predicted_scores_epoch1'})
combined_set['predicted_scores_epoch2'] = epoch2['predicted_scores']
combined_set['predicted_scores_epoch3'] = epoch3['predicted_scores']

## Calculate Cartography Measures

In [7]:
def softmax(x):
    """Calculates the softmax of a list of numbers."""
    e_x = np.exp(x - np.max(x))  # Subtracting max for numerical stability
    return e_x / e_x.sum()

def calculate_confidence(label,epoch1,epoch2,epoch3):


    epoch1_sm = softmax(epoch1)
    epoch2_sm = softmax(epoch2)
    epoch3_sm = softmax(epoch3)

    output_softmax = [epoch1_sm[label],epoch2_sm[label],epoch3_sm[label]]
    confidence = sum(output_softmax) / len(output_softmax)
    variability = np.sqrt(np.sum(np.square(output_softmax - confidence)) / 3)
    return (confidence,variability)


combined_set['confidence'] = combined_set.apply(lambda x: calculate_confidence(x['label'],x['predicted_scores_epoch1'],x['predicted_scores_epoch2'],x['predicted_scores_epoch3'])[0], axis = 1)
combined_set['variability'] = combined_set.apply(lambda x: calculate_confidence(x['label'],x['predicted_scores_epoch1'],x['predicted_scores_epoch2'],x['predicted_scores_epoch3'])[1], axis = 1)

In [9]:
training_set_high_variability = combined_set.sort_values(by = 'variability', ascending = False).head(n = math.ceil(len(combined_set) / 3))
training_set_hard_to_learn = combined_set.sort_values(by = 'confidence').head(n = math.ceil(len(combined_set) / 3))
training_set_easy_to_learn = combined_set.sort_values(by = 'confidence',ascending = True).head(n = math.ceil(len(combined_set) / 3))
training_set_random = combined_set.sample(frac = 0.3333)

In [11]:
training_set_high_variability[['premise','hypothesis','label','confidence','variability']].to_json('training_high_var.jsonl',orient = 'records',lines = True)
training_set_hard_to_learn[['premise','hypothesis','label','confidence','variability']].to_json('training_hard_to_learn.jsonl',orient = 'records',lines = True)
training_set_easy_to_learn[['premise','hypothesis','label','confidence','variability']].to_json('training_easy_to_learn.jsonl',orient = 'records',lines = True)
training_set_random[['premise','hypothesis','label','confidence','variability']].to_json('training_random.jsonl',orient = 'records',lines = True)

'A lady with a black shirt, purple dress, and pink award bow opening gifts.'